* [模型简介](#模型简介)
* [快速开始](#快速开始)
  * [通用参数释义](#通用参数释义)
  * [自然语言推断任务](#自然语言推断任务)
* [参考论文](#参考论文)

## 1. ERNIE-M模型简介


[ERNIE-M](https://arxiv.org/abs/2012.15674) 是百度提出的一种多语言语言模型。原文提出了一种新的训练方法，让模型能够将多种语言的表示与单语语料库对齐，以克服平行语料库大小对模型性能的限制。原文的主要想法是将回译机制整合到预训练的流程中，在单语语料库上生成伪平行句对，以便学习不同语言之间的语义对齐，从而增强跨语言模型的语义建模。实验结果表明，ERNIE-M 优于现有的跨语言模型，并在各种跨语言下游任务中提供了最新的 SOTA 结果。

本项目是 ERNIE-M 的 PaddlePaddle 动态图实现， 包含模型训练，模型验证等内容。

## 2. 模型效果及应用场景

### 2.1 自然语言推断任务

#### 2.1.1 数据集

XNLI 是 MNLI 的子集，并且已被翻译成14种不同的语言（包含一些较低资源语言）。与 MNLI 一样，目标是预测文本蕴含（句子 A 是否暗示/矛盾/都不是句子 B ）。


# 3. 模型如何使用

## 3.1 模型训练

* 安装PaddleNLP包

In [ ]:
!pip install paddlenlp
!pip install datasets

* 下载

（不在Jupyter Notebook上运行时需要将"！"或者"%"去掉。）

In [ ]:
# 克隆PaddleNLP（从gitee上更快）
!wget https://gitee.com/paddlepaddle/PaddleNLP/blob/develop/model_zoo/ernie-m/run_classifier.py

* 单卡训练

In [ ]:
# 设置GPU
%env CUDA_VISIBLE_DEVICES=0
!python run_classifier.py \
    --task_type cross-lingual-transfer \
    --batch_size 16 \
    --model_name_or_path ernie-m-base \
    --save_steps 12272 \
    --output_dir output

* 多卡训练

In [ ]:
# 设置GPU
%env CUDA_VISIBLE_DEVICES=0
!python -m paddle.distributed.launch --gpus 0,1 --log_dir output run_classifier.py \
    --task_type cross-lingual-transfer \
    --batch_size 16 \
    --model_name_or_path ernie-m-base \
    --save_steps 12272 \
    --output_dir output

### 3.3 通用参数释义

- `task_type` 表示了自然语言推断任务的类型，目前支持的类型为："cross-lingual-transfer", "translate-train-all"
  ，分别表示在英文数据集上训练并在所有15种语言数据集上测试、在所有15种语言数据集上训练和测试。
- `model_name_or_path` 指示了 Fine-tuning 使用的具体预训练模型以及预训练时使用的tokenizer，目前支持的预训练模型有："ernie-m-base"， "ernie-m-large"
  。若模型相关内容保存在本地，这里也可以提供相应目录地址，例如："./checkpoint/model_xx/"。
- `output_dir` 表示模型保存路径。
- `max_seq_length` 表示最大句子长度，超过该长度将被截断，不足该长度的将会进行 padding。
- `memory_length` 表示当前的句子被截取作为下一个样本的特征的长度。
- `learning_rate` 表示基础学习率大小，将于 learning rate scheduler 产生的值相乘作为当前学习率。
- `num_train_epochs` 表示训练轮数。
- `logging_steps` 表示日志打印间隔步数。
- `save_steps` 表示模型保存及评估间隔步数。
- `batch_size` 表示每次迭代**每张**卡上的样本数目。
- `weight_decay` 表示AdamW的权重衰减系数。
- `layerwise_decay` 表示 AdamW with Layerwise decay 的逐层衰减系数。
- `adam_epsilon` 表示AdamW优化器的 epsilon。
- `warmup_proportion` 表示学习率warmup系数。
- `max_steps` 表示最大训练步数。若训练`num_train_epochs`轮包含的训练步数大于该值，则达到`max_steps`后就提前结束。
- `seed` 表示随机数种子。
- `device` 表示训练使用的设备, 'gpu'表示使用 GPU, 'xpu'表示使用百度昆仑卡, 'cpu'表示使用 CPU。
- `use_amp` 表示是否启用自动混合精度训练。
- `scale_loss` 表示自动混合精度训练的参数。

## 4. 模型原理

原文提出两种方法建模各种语言间的对齐关系:

- **Cross-Attention Masked Language Modeling(CAMLM)**: 该算法在少量双语语料上捕捉语言间的对齐信息。其需要在不利用源句子上下文的情况下，通过目标句子还原被掩盖的词语，使模型初步建模了语言间的对齐关系。
- **Back-Translation masked language modeling(BTMLM)**: 该方法基于回译机制从单语语料中学习语言间的对齐关系。通过CAMLM 生成伪平行语料，然后让模型学习生成的伪平行句子，使模型可以利用单语语料更好地建模语义对齐关系。


![framework](./framework.png)

## 5. 注意事项



## 6. 相关论文以及引用信息

```bibtex
@article{Ouyang2021ERNIEMEM,
  title={ERNIE-M: Enhanced Multilingual Representation by Aligning Cross-lingual Semantics with Monolingual Corpora},
  author={Xuan Ouyang and Shuohuan Wang and Chao Pang and Yu Sun and Hao Tian and Hua Wu and Haifeng Wang},
  journal={ArXiv},
  year={2021},
  volume={abs/2012.15674}
}
```